Nesta etapa iniciaremos a **Fase 3.1** do plano de atividades, que corresponde à primeira ação prática com dados reais: a **ingestão do conjunto de dados** que será usado para treinamento e rastreamento de experimentos. A partir deste ponto, todas as operações ocorrerão **dentro de notebooks Jupyter** abertos pelo VS Code no DevContainer, seguindo o protocolo estabelecido para manter **rastreabilidade, reprodutibilidade** e **versionamento coerente**.

O objetivo desta célula é **criar o diretório de dados brutos**, baixar o arquivo de entrada diretamente na pasta correta e garantir que os arquivos possam ser versionados pelo DVC logo em seguida. Essa prática elimina discrepâncias entre ambiente local e ambiente de produção, pois o download ocorre **dentro do container**, conectado à mesma rede do Compose.

